# Reflection Pattern

The first pattern we are going to implement is the **reflection pattern**. 

---

<img src="./images/reflection.png" alt="Alt text" width="600"/>

---

This pattern allows the LLM to reflect and critique its outputs, following the next steps:

1. The LLM **generates** a candidate output. If you look at the diagram above, it happens inside the **"Generate"** box.
2. The LLM **reflects** on the previous output, suggesting modifications, deletions, improvements to the writing style, etc.
3. The LLM modifies the original output based on the reflections and another iteration begins ...

**Now, we are going to build, from scratch, each step, so that you can truly understand how this pattern works.**

## Generation Step

The first thing we need to consider is:

> What do we want to generate? A poem? An essay? Python code?

For this example, I've decided to test the Python coding skills of Llama3 70B (that's the LLM we are going to use for all the tutorials). In particular, we are going to ask our LLM to code a famous sorting algorithm: **Merge Sort**. 

---

<img src="../img/mergesort.png" alt="Alt text" width="500"/>

### Groq Client and relevant imports

In [1]:
import os
from pprint import pprint
from groq import Groq
from dotenv import load_dotenv
from IPython.display import display_markdown

# Remember to load the environment variables. You should have the Groq API Key in there :)
load_dotenv()

client = Groq()

We will start the **"generation"** chat history with the system prompt, as we said before. In this case, let the LLM act like a Python 
programmer eager to receive feedback / critique by the user.

In [2]:
generation_chat_history = [
    {
        "role": "system",
        "content": "You are a Python programmer tasked with generating high quality Python code."
        "Your task is to Generate the best content possible for the user's request. If the user provides critique," 
        "respond with a revised version of your previous attempt."
    }
]

Now, as the user, we are going to ask the LLM to generate an implementation of the **Merge Sort** algorithm. Just add a new message with the **user** role to the chat history.

In [3]:
generation_chat_history.append(
    {
        "role": "user",
        "content": "Generate a Python implementation of the Merge Sort algorithm"
    }
)

Let's generate the first version of the essay.

In [4]:
mergesort_code = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

generation_chat_history.append(
    {
        "role": "assistant",
        "content": mergesort_code
    }
)

In [5]:
display_markdown(mergesort_code, raw=True)

Here is a Python implementation of the Merge Sort algorithm:
```
def merge_sort(arr):
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: The array to be sorted
    :return: The sorted array
    """
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left, right):
    """
    Merges two sorted arrays into a single sorted array.

    :param left: The first sorted array
    :param right: The second sorted array
    :return: The merged sorted array
    """
    result = []
    while len(left) > 0 and len(right) > 0:
        if left[0] <= right[0]:
            result.append(left.pop(0))
        else:
            result.append(right.pop(0))
    result.extend(left)
    result.extend(right)
    return result


# Example usage:
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
This implementation uses a top-down approach, recursively dividing the input array into smaller subarrays until each subarray has only one element, and then merging the subarrays back together in a sorted manner.

Let me know if you have any feedback or suggestions for improvement!

## Reflection Step

Now, let's allow the LLM to reflect on its outputs by defining another system prompt. This system prompt will tell the LLM to act as Andrej Karpathy, computer scientist and Deep Learning wizard.

>To be honest, I don't think the fact of acting like Andrej Karpathy will influence the LLM outputs, but it was fun :)

<img src="../img/karpathy.png" alt="Alt text" width="500"/>

In [6]:
reflection_chat_history = [
    {
    "role": "system",
    "content": "You are Andrej Karpathy, an experienced computer scientist. You are tasked with generating critique and recommendations for the user's code",
    }
]

The user message, in this case,  is the essay generated in the previous step. We simply add the `mergesort_code` to the `reflection_chat_history`.

In [7]:
reflection_chat_history.append(
    {
        "role": "user",
        "content": mergesort_code
    }
)

Now, let's generate a critique to the Python code.

In [8]:
critique = client.chat.completions.create(
    messages=reflection_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [9]:
display_markdown(critique, raw=True)

Overall, your implementation of the Merge Sort algorithm is correct and well-documented. Here are some minor suggestions for improvement:

1. **Consistent naming conventions**: You're using both camelCase and underscore notation for variable and function names. To maintain consistency, I'd recommend sticking to Python's convention of using underscore notation (e.g., `merge_sort` instead of `mergeSort`).

2. **Type hints and annotations**: While you've documented the functions nicely, consider adding type hints and annotations to explicitly specify the function parameters and return types. For example, `def merge_sort(arr: list) -> list:`.

3. **Input validation**: Currently, the implementation assumes that the input `arr` is a list. Consider adding a simple input validation check to ensure that `arr` is indeed a list. You can raise a `TypeError` if the input is not a list.

4. **docstring formatting**: The docstrings look great, but you can improve the formatting by using proper reStructuredText syntax. For example, you can use `:param type name:` instead of `:param name: The ...`.

5. **merge function optimization**: In the `merge` function, you can optimize the merging process by using a single list comprehension instead of iteratively appending elements to the `result` list.

Here's the refactored code incorporating these suggestions:
```
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: list
        The array to be sorted
    :return: list
        The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: list
        The first sorted array
    :param right: list
        The second sorted array
    :return: list
        The merged sorted array
    """
    return [x for pair in zip 长((x for x in left if x <= right[0]), (x for x in right if x <= left[0])) for x in pair] + left[len(list((x for x in left if x <= right[0]))):] + right[len(list((x for x in right if x <= left[0]))):]


# Example usage:
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
These changes are minor, and your implementation is already quite good. The optimized `merge` function uses list comprehensions, which can be more efficient than iterative appending. However, it's worth noting that the optimization may not have a significant impact on performance in this case, as the dominant factor is still the recursive splitting and merging process.

Finally, we just need to add this *critique* to the `generation_chat_history`, in this case, as the `user` role.

In [10]:
generation_chat_history.append(
    {
        "role": "user",
        "content": critique
    }
)

## Generation Step (II)

In [11]:
essay = client.chat.completions.create(
    messages=generation_chat_history,
    model="llama3-70b-8192"
).choices[0].message.content

In [12]:
display_markdown(essay, raw=True)

Thank you for the thorough review and suggestions! I appreciate the feedback and will definitely incorporate the changes into my implementation.

Here's the revised code:
```
def merge_sort(arr: list) -> list:
    """
    Sorts an array using the Merge Sort algorithm.

    Time complexity: O(n log n)
    Space complexity: O(n)

    :param arr: list
        The array to be sorted
    :return: list
        The sorted array
    """
    if not isinstance(arr, list):
        raise TypeError("Input must be a list")
    if len(arr) <= 1:
        return arr

    mid = len(arr) // 2
    left = arr[:mid]
    right = arr[mid:]

    left = merge_sort(left)
    right = merge_sort(right)

    return merge(left, right)


def merge(left: list, right: list) -> list:
    """
    Merges two sorted arrays into a single sorted array.

    :param left: list
        The first sorted array
    :param right: list
        The second sorted array
    :return: list
        The merged sorted array
    """
    result = []
    i, j = 0, 0
    while i < len(left) and j < len(right):
        if left[i] <= right[j]:
            result.append(left[i])
            i += 1
        else:
            result.append(right[j])
            j += 1
    result.extend(left[i:])
    result.extend(right[j:])
    return result


# Example usage:
arr = [5, 2, 8, 3, 1, 4, 6]
arr = merge_sort(arr)
print(arr)  # [1, 2, 3, 4, 5, 6, 8]
```
I've addressed all the points you mentioned:

1. Consistent naming conventions: I've used underscore notation for function and variable names.
2. Type hints and annotations: I've added type hints and annotations for function parameters and return types.
3. Input validation: I've added a check to ensure the input `arr` is a list, and raise a `TypeError` if it's not.
4. docstring formatting: I've reformatted the docstrings using proper reStructuredText syntax.
5. merge function optimization: I've implemented a more efficient merging process using a single loop and list extension, rather than iteratively appending elements to the result list.

However, I didn't implement the list comprehension optimization you suggested, as it seemed less readable and more complex than the revised `merge` function I provided.

Thank you again for the feedback!

## And the iteration starts again ...

After **Generation Step (II)** the corrected Python code will be received, once again, by Karpathy. Then, the LLM will reflect on the corrected output, suggesting further improvements and the loop will go, over and over for a number **n** of total iterations.

> There's another possibility. Suppose the Reflection step can't find any further improvement. In this case, we can tell the LLM to output some stop string, like "OK" or "Good" that means the process can be stopped. However, we are going to follow the first approach, that is, iterating for a fixed number of times.